# Training Deep Neural Networks

What if you need to tackle a very complex problem, such as detecting hundreds of types of objects in high-resolution images? You may need to train a much deeper DNN, perhaps with 10 layers or much more, each containing hundreds of neurons, connected by hundreds of thousands of connections (and weights). This would not be a walk in the park:

- First, you would be faced with the tricky *vanishing gradients* problem (or the related *exploding gradients* problem) that affects deep neuroal networks and makes lower layers very hard to train.

- Second, you might not have enough training data for such a large network, or it might be too costly to label.

- Third, training might be extremely slow.

- Fourth, a model with millions of parameters would severely risk overfitting the training set, especially if there are not enough training instances, or they are too noisy.

In this chapter, we will go through each of these problems in turn and present techniques to solve them.

## Vanishing/Exploding Gradients Problems

As we discussed, the backprop algo works by going from the output layer to the input layer, propagating the error gradient on the way. Once the algo has computed the gradient of the cost function with regards to each parameter in the network, it uses these gradients to update each parameter with a Gradient Descent step.

Unfortunately, gradients often get smaller and smaller as the algo progresses down to the lower layers. As a result, the GD update leaves the lower layer connection weights virtually unchanged, and training never converges to a good solution. This is called the *vanishing gradient* problem. In some cases, the opposite can happen: the gradients can grow bigger and bigger, so many layers get insanely large weight updates and he algo diverges. This is the *exploding gradients* problem which is mostly encountered in recurrent neural networks. 

More generally, DNN suffer from unstable gradients; different layers may learn at widely different speeds.

Although this unfortunate behavior has been empirically observed for quite a while (it was one of the reasons why deep neural networks were mostly abandoned for a long time), it is only around 2010 that significant progress was made in understanding it. A paper by Glorot and Bengio found a few suspects, including the combination of the popular logistic sigmoid activation funciton and the weight initialization technique that was most popular at the time (random init. using a normal distribution with mean 0 and std 1). In short, they showed that with this activation function and this initialization scheme, the variance of the outputs of each layer is much greater than the variance of its inputs. Going forward in the network, the variance keeps increasing after each layer until the activation function saturates at the top layers. This is actually made worse by the fact that the logistic function hass a mean of 0.5 and not of 0.

Looking at the logistic activation function (see figure below), you can see that when inputs become large (negative or positive), the function saturates at 0 or 1, with a derivative extremely close to 0. Thus when backprop kicks in, it has virtually no gradient to propagate back through the network, and what little gardient exists keeps getting diluted as backprop progresses down through the top layers, so there is really nothing left for the lower layers.

![](logisticfun.png)

## Glorot and He Initialization

In their paper, Glorot and Bengio propose a way to significantly alleviate this problem. We need the signal to flow properly in both directions: in the forward direction when making predictions, and in the reverse direction when backpropagating gradients. We don't want the signal to die out, nor do we want it to explode and saturate. 

For the signal to flow properly, the authors argue that we need the variance of the outputs of each layer to be equal to the variance of its inputs, and we also need the gradients to have equal variance before and after flowing through a layer in the reverse direction.

`Here's an analogy: if you set a microphone amplifier's knob too close to zero, people won't hear your voice, but if you set it too close to the max, your voice will be saturated and people won't understand what you are saying. Now imagine a chain of such amplifiers: they all need to be set properly in order for your voice to come out loud and clear at the end of the chain. Your voice has to come out of each amplifier at the same amplitude as it came in.`

It is actually not possible to guarantee both unless the layer has an equal number of inputs and neurons (these numbers are called the *fan-in* and *fan-out* of the layer), but they proposed a good compromise that has proven to work very well in practice: the connection weights of each layer must be initialized randomly as described in the equation below, where $fan_{avg} = (fan+{in} + fan_{out})/2$. This initialization strategy is called *Xavier initialization* or *Glorot initialization*.

$$ \text{Normal distribution with mean }0\text{ and variance }\sigma^2 = \frac{1}{fan_{avg}}$$

$$ \text{Or a uniform distribution between }-r\text{ and }+r\text{, with } r = \sqrt{\frac{3}{fan_{avg}}}$$

If you just replace $fan_{avg}$ with $fan_{in}$ in the equation above, you get an initialization strategy that was was actually already proposed by Yann LeCun in the 1990s. It is equivalent to Glorot initialization when $fan_{in} = fan_{out}$. It took over a decade for researchers to realize just how important this trick really is. Using Glorot initialization can speed up training considerably, and it is one of the tricks that led to the current success of Deep Learning.

Some papers have provided similar strategies for different activation functions. These strategies differ only by the scale of the variance and whether they use $fan+{avg}$ or $fan_{in}$ as shown in the table below (for the uniform distribution, just compute $r = \sqrt{3\sigma^2}$. The initialization strategy for the ReLU activation function (and its variants, including the ELU activation described shortly) is sometimes called *He initialization*. The SELU activation function will be explained later in this chapter. It should be used with LeCun initialization (preferably with a normal distribution, as we will see).

|Initialization|Activation functions|$\sigma^2$(Normal)|
|--------------|--------------------|------------------|
|Glorot|None, Tanh, Logistic, Softmax|1/$fan_{avg}$|
|He|ReLU & variants|2/$fan_{in}$|
|LeCun|SELU|1/$fan_{in}$|

By default, Keras uses Glorot initialization with a uniform distibution. You can change this to He initialization by setting `kernel_initializer="he_uniform"` or `kernel_initializer="he_normal"` when creating a layer, like this:

In [2]:
from tensorflow import keras

In [3]:
keras.layers.Dense(10, activation = "relu", kernel_initializer="he_normal")

If you want He initialization with a uniform distribution, but based on $fan_{avg}$ rather than $fan_{in}$ you can use the `VarianceScaling` initializer like this:

In [4]:
he_avg_init = keras.initializers.VarianceScaling(scale=2, 
                                                 mode='fan_avg',
                                                distribution='uniform')
keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)

## Nonsaturating Activation Functions

One of the insights in the 2010 paper by Glorot and Bengio was that the vanishing/ exploding gradients problems were in part due to a poor choice of activation function. Until then most people had assumed that if Mother Nature had chosen to use roughly sigmoid activation functions in biological neurons, they must be an excellent choice. But it turns out that other activation functions behave much better in deep neural networks, in particular the ReLU activation function, mostly because it does not saturate for positive values (and also because it is quite fast to compute). 

Unfortunately, the ReLU activation function is not perfect. It suffers from a problem known as the dying ReLUs: during training, some neurons effectively die, meaning they stop outputting anything other than 0. In some cases, you may find that half of your network’s neurons are dead, especially if you used a large learning rate. A neuron dies when its weights get tweaked in such a way that the weighted sum of its inputs are negative for all instances in the training set. When this happens, it just keeps outputting 0s, and gradient descent does not affect it anymore since the gradient of the ReLU function is 0 when its input is negative.

To solve this problem, you may want to use a variant of the ReLU function, such as the *leaky ReLU*. This function is defined as $LeakyReLU_{a}(z) = max(\alpha z,z)$ (see figure below).

![](leakyrelu.png)

The hyperparameter $\alpha$ defines how much the function "leaks": it is the slope of the function for $z<0$, and is typically set to 0.01. This small slope ensures that leaky ReLUs never die; they can go into a log coma, btthey have a chance to eventually wake up. A 2015 paper compared several variants of the ReLU activation function and one of its conclusions was that the leaky variants always outperformed the strict ReLU activation function. In fact, setting $\alpha = 0.2$ (huge leak) seemed to result in better performance than $\alpha = 0.01$ (small leak). They also evaluated the *randomized leaky ReLU* (RReLU), where $\alpha$ is picked randomly in a given range during training, and it is fixed to an average value during testing. It also performed fairly well and seemed to act as a regularizer (reducing the risk of overfitting the training set).

Finally, they also evaluated the *parameric leaky ReLU* (PReLU), where $\alpha$ is authorized to be learned during training (instead of being and hyperparameter, it becomes a parameter that can be modified by backprop). This was reported to strongly outperform ReLU on large image datasets, but on smaller datasets it runs the risk of overfitting the training set.

Last but not least, a 2015 paper by Clevert et al. proposed a new activation function called the *exponential linear unit* (ELU) that outperformed all ReLU variants in the experiments: training time was reduced and the neural network performed better on the test set. See the equation and figure below.

$$ ELU_{ \alpha}(z) \begin{cases}
\alpha (exp(z)) \ \text{ if } z < 0 \\
z \ \text{if} z \geq 0
\end{cases}
$$

![](ELU.png)

It looks a lot like the ReLU function, with a few major differences:

- First it takes on negative values when $z<0$, which allows the unit to have an average output closer to 0. This helps alleviate the vanishing gradients problem, as discussed earlier. The hyperparameter $\alpha$ defines the value that the ELU function approaches when $z$ is a large negative number. It is usually set to 1, but you can tweak it like any other hyperparameter if you want.

- Second, it has a nonzero gradient for $z <0$, which avoids the dead neurons problem.

- Third, if $\alpha$ is equal to 1 then the function is smooth everywhere, including around $z=0$, which helps speed up Gradient Descent, since it does not bounce as much left and right of $z=0$.

The main drawback of the ELU function is that is slowe to compute than the ReLU and its variants, but during training this is compensated by the faster convergence rate. However, at test time an ELU network will be slower than a ReLU network.

Moreover, in a 2017 paper by Klambauer et al., called "Self Normalizing Neural Networks", the authors showed that if you build a neural network composed exclusively of a stack of dense layers, and if all hidden layers use the SELU activation function (which is just a scaled version of the ELU act. fun.), then the network will *self-normalize*: the output of each layer will tend to preserve mean 0 and standard deviation 1 during training, which solves the vanishing/exploding gradients problem. As a result, this activation function often outperforms the other act. funs very significantly for such neural nets (especially deep ones). However, there are a few conditions for self-normalization to happen:

- The input features must be standardized (mean 0 and stdev 1).

- Every hidden layer's weights must also be initialized using LeCun normal initialization. In Keras, this means setting `kernel_initializer="lecun_normal"`.

- The network's architecture must be sequential. Unfortunately, if you try to use SELU in non-sequential architectures, such as recurrent networks or networks with *skip-connections* (such as in wide & deep nets), self-normalization will not be guaranteed, so SELU will not necessarily outperform other act. funs.

- The paper only guarantees self-normalization if all layers are dense. However, in practice the SELU actviation function seems to work great with convolutional neuroal nets as well.

**Note**:

So which activation function should you use for the hidden layers of your DNN? Although your mileage will vary, in general SELU > ELU > leaky ReLU (and variants) > ReLU > tanh > logistic. If the network's architecture prevents it from self-normalizing, then ELU may perform better than SELU (since SELU is not smooth at $z = 0$). If you care a lot about runtime latency, then you may prefer leaky ReLU. If you don't want to tweak yet another hyperparameter, you may just use the default $\alpha$ values used by Keras (e.g. for the leaky ReLU). If you have spare time and computing power, you can use cross-validation to evaluate other activation functions, in particular RReLU if your network is overfitting, or PReLU if you have a huge training set.

To use the leaky ReLU activation function, you must create a `LeakyReLU` instance like this:

In [5]:
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
layer = keras.layers.Dense(10, activation=leaky_relu,
                          kernel_initializer="he_normal")

For PReLU, just replace `LeakyRelu(alpha=0.2)` with `PReLU()`. There is currently no official implementation of RReLU in Keras, but you can fairly easily implement your own.

For `SELU` activation, just set `activation="selu"` and `kernel_initializer="lecun_normal"` when creating a layer:

In [6]:
layer = keras.layers.Dense(10, activation="selu",
                          kernel_initializer="lecun_normal")

## Batch Normalization

Although using He initialization along with ELU (or any variant of ReLU) can significantly reduce the vanishing/exploding gradients problems at the beginning of training, it doesn't guarantee that they won't come back during training.

In a 2015 paper, Ioffe and Szegedy proposed a technique called *Batch Normalization* (BN) to address the vanishing/exploding gradients problems.

The technique consists of adding an operation in the model just before or after the activation function of each hidden layer, simply zero-centering and normalizing each input, then scaling and shifting the result using two new parameter vectors per layer: one for scaling, the other for shifting. In other words, this operation lets the model learn the optimal scale and mean of each of the layer's inputs. In many cases, if you add a BN layer as the very first layer of your neural network, you do not need to standardize your training set (e.g. using `StandardScaler`): the BN layer will do it for you (well, approximately, since it only looks at one batch at a time, and it can also rescale and shift each input feature).

In order to zero-center and normalize the inputs, the algo needs to estimate each input's mean and standard deviation. It does so by evaluating the mean and standard deviation of each input over the current mini-batch (hence the name "Batch Normalization"). The whole operation is summarized in the equation below:

1. $\mu_{B} = \frac{1}{m_B}\sum^{m_B}_{i=1}x^{(i)}$

2. $\sigma^2_{B} = \frac{1}{m_B}\sum^{m_B}_{i=1}(x^{(i)}-\mu_B)^2$

3. $\hat{x}^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt{\sigma^2_B + \epsilon}}$

4. $z^{(i)} = \gamma \otimes \hat{x}^{(i)} + \beta$



- $\mu_B$ is the vector of input means, evaluated over the whole mini-batch B (it contains one mean per input)

- $\sigma_B^2$ is the vector of input standard deviations, also evaluated over the whole mini-batch (it contains one standard deviation per input).

- $m_B$ is the number of instances in the mini-batch.

- $\hat{x}^{(i)}$ is the vector of zero-centered and normalized inputs for instance i.

- $\gamma$ is the output scale parameter vector for the layer (it contains one scale parameter per input).

- $\otimes$ represents element-wise multiplication (each input is multiplied by its corresponding output scale parameter).

- $\beta$ is the output shift (offset) parameter vector for the layer (it contains one offset parameter per input). Each input is offset by its corresponding shift parameter.

- $\epsilon$ is a tiny number to avoid division by zero (typically $10^{-5}$). This is called a *smoothing term*.

- $z^{(i)}$ is the output of the BN operation: it is a rescaled and shifted version of the inputs.

So during training, BN just standardizes its inputs then rescales and offsets them. What about at test time? Well it is not that simple. Indeed, we may need to make predictions for individual instances rather than for batches of instances: in this case, we will have no way to compute each input's mean and stdev. Moreover, even if we do have a batch of instances, it may be too small, or the instances may not be independent and identically distributed (IID), so computing statistics over the batch would be unreliable (during training, the batches should not be too small, if possible more than 30 instances, and all instances should be IID). One solution could be to wait until the end of training, then run the whole training set through the neural network, and compute the mean and standard deviation of each input of the BN layer. 

These "final" input means and stdev can then be used instead of the batch input means and stdev when making predictions. However, it is often preferred to estimate these final statistics during training using a moving average of the layer's input means and standard deviations. 

To sum up, four parameter vectors are learned in each batch-normalized layer: $\gamma$ (the output scale vector) and $\beta$ (the output offset vector) are learned through regular backprop, and $\mu$ (the final input mean vector), and $\sigma$ (the final input stdev vector) are estimated using an exponential moving average. Note that $\mu$ and $\sigma$ are estimated during training, but they are not used at all during training, only after training (to replace the batch input means and standard deviations in the equation above.

The author demonstrated that this technique considerably improved all the DNN they experimented with, leading to a huge improvement in the ImageNet classification task. The vanishing gradients problem was strongly reduced, to the point that they could use saturating activation functions such as tanh and even logistic function. The networks were also much less sensitive to the weight initialization. They were able to use much larger learning rates, significantly speeding up the learning process. Finally, BN also acts like a regularizer, reducing the need for other regularization techniques (such as dropout, described later in this chapter).

BN does, however, add some complexity to the model (although it can remove the need for normalizing the input data, as we discussed earlier). Moreover, there is a runtime penalty: the neural network makes slower predictions due to the extra computations required at each layer. So if you need predictions to be lightning-fast, you may want to check how well plain ELU + He initialization performs before playing with BN.

**Note**:

You may find that training is rather slow, because each epoch takes much more time when you use batch normalization. However, this is usually counterbalanced by the fact that convergence is much faster with BN, so it will take fewer epochs to reach the same performance. All in all, *wall time* will usually be smaller (this is the time measured by the clock on your wall).

### Implementing Batch Normalization with Keras

As with most things with Keras, implementing Batch Normalization is quite simple. Just add a `BatchNormalization` layer before or after each hidden layer's activation function, and optionally add a BN layer as well as the first layer in your model. 

For example, this model applies BN after every hidden layer and as the first layer in the model (after flattening the input images):

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")    
])

W0902 16:15:15.947628 13968 deprecation.py:506] From C:\Users\levka\Anaconda3\envs\ds001\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


That's all! In this tiny example with just two hidden layers, it's unlikely that BN will have a very positive impact, but for deeper networks it can make a tremendous difference.

If you display the model summary, you can see that each BN layer adds 4 parameters per input: $\gamma, \beta, \mu $ and $\sigma$. The last two parameters, $\mu$ and $\sigma$, are the moving averages, they are not affected by backprop, so Keras call them "Nontrainable".

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_5 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1

Let's look at the parameters of the first BN layer. Two are trainable (by backprop), and two are not (estimated during training, unaffected by backprop):

In [9]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

Now when you create a BN layer in Keras, it also creates two operations that will be called by Keras at each iteration during training. These operations will update the moving averages. Since we are using the TensorFlow backend, these operations are TensorFLow operations.

In [10]:
model.layers[1].updates

[<tf.Operation 'batch_normalization/cond_2/Merge' type=Merge>,
 <tf.Operation 'batch_normalization/cond_3/Merge' type=Merge>]

The authors of the BN paper argued in favor of adding the BN layers before the activation functions, rather than after (as we just did). There is some debate about this, as it seems to depend on the task. So that's one more thing that you can experiment with to see which option works best on your dataset. To add the BN layers before the activation functions, we must remove the activation function from the hidden layers, and add them as separate layers after the BN layers. Moreover, since a BN layer includes one offset parameter per input, you can remove the bias term from the previous layer (just pass `use_bias=False` when creating it).

In [11]:
model = keras.models.Sequential([
keras.layers.Flatten(input_shape=[28, 28]),
keras.layers.BatchNormalization(),
keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
keras.layers.BatchNormalization(),
keras.layers.Activation("elu"),
keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
keras.layers.Activation("elu"),
keras.layers.BatchNormalization(),
keras.layers.Dense(10, activation="softmax")
])

The `BatchNormalization` class has quite a few hyperparameters you can tweak. The defaults will usually be fine, but you may occasionally need to tweak the `momentum`. This hyperparameter is used when updating the exponential moving averages: given a new value $\mathrm{v}$ (i.e. a new vector of input means or stdevs computed over the current batch), the running average $\hat{\mathrm{v}}$ is updated using the following equation:

$\hat{\mathrm{v}} \gets \hat{\mathrm{v}} \times \text{momentum} + \mathrm{v} \times (1 - \text{momentum})$

A good momentum value is typically close to 1 -- for example, 0.9, 0.99, or 0.999 (you want more 9s for larger datasets and smaller mini-batches).

Another important hyperparameter is `axis`: it determines which axis should be normalized. It defaults to -1, meaning that by default it will normalize the last axis (using the means and stdevs computed across the *other* axes). For example, when the input batch is 2D (i.e. the batch shape is \[ batch size, features \]), this means that each input feature will be normalized based on the mean and standard deviation computed across all the instances in the batch. For example, the first BN layer in the previous code example will independently normalize (and rescale and shift) each of the 784 input features. However, if we move the first BN layer before the `Flatten` layer, then the input batches will be 3D, with shape \[batch size, height, width\], therefore the BN layer will compute 28 means and 28 stdevs (one per column of pixels, computed across all instances in the batch, and all rows in the column), and it will normalize all pixels in a given column using the same mean and stdev. There will also be just 28 scale parameters and 28 shift parameters. If instead you still want to treat each of the 784 pixels independently, then you should set `axis=[1, 2]` (height and width).

Notice that the BN layer does not perform the same computation during training and after training: it uses batch statistics during training, and the "final" statistics after training (i.e. the final value of the moving averages). Let's take a peek at the source code of this class to see how this is handled:

![](batchnorm.png)

The `call()` method is the one that actually performs the computations, and as you can see it has extra an `training` argument: if it is `None` it falls back to `keras.backend.learning_phase()`, which returns 1 during training (the `fit()` method  ensures that). Otherwise, it returns 0. If you ever need to write a custom layer, and it needs to behave differently during training and testing, simply use the same pattern.

BN has become one of the most used layers in DNN, to the point that it is often omitted in the diagrams, as it is assumed that BN is added after every layer. However, a very recent paper by Zhang et al. may well change this: the authors show that by using a novel fixed-update (fixup) weight initialization technique, they manage to train a very DNN (10,000 layers) without BN, achieving state-of-the-art performance on complex image classification tasks.

## Gradient Clipping 338/360